In [ ]:
#Lab 1 - Agent-Based Model simulation with simulated foraging ants
#I think the ants follow a fairly expected behavior with this model that somewhat resembles actual behavior.

#Key take-aways:
# - The ants move with persistance around the space, generally following a tack they've been on but not exclusively.
# - If the ants come across food, they "take it" (the food sprite disappears from the space) and head to the nest, dropping
#pheromones every 25 pixels on their way back.
# - Ants who have finished dropping off food at the nest, or ants who have come across a pheromone trail along their
#path generally follow the pheromones away from the nest. They do not typically make a "bee-line" up the pheromone trail. 
#They more gradually follow it and if the trail is disappearing (the pheromones have a lifespan), the ants are only
#interested for a brief time before continuing a persitance movement. This behavior leads to somewhat gridded areas of
#pheromones around food clusters and does seem to draw ants out to further clusters. The ants also don't exclusively follow
#a single pheromone path all the time which I believe more closely mimics their behavior.

#Noted performance issues:
# - Frame rate goes down significantly when a certain amount of pheromones have been added to the simulation. Turning off
#the pheromones visualization does not improve performance.
# - The ants will occassionaly pick up more than one piece of food.

In [1]:
import pygame, sys, random as r, numpy as np, math, operator as opr

#--Color definition--
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)
BLACK = (0, 0, 0)
PURPLE = (255, 0, 255)

#--Class definitions--
class Nest(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface([20,20])
        self.rect = self.image.get_rect()

class Food(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface([5,5])
        self.image.fill((0,128,0))
        self.rect = self.image.get_rect() 

class Pheromone(pygame.sprite.Sprite):
    def __init__(self, id, nest_rect, pos_x, pos_y):
        super().__init__()
        self.image = pygame.Surface([4,4])
        self.image.fill(PURPLE)
#         self.image.set_colorkey(PURPLE)
        self.rect = self.image.get_rect()
        self.timer = 600
        self.id = id
        self.rect.x = pos_x
        self.rect.y = pos_y
        self.distance = math.hypot(self.rect.x - nest_rect.center[0], self.rect.y - nest_rect.center[1])
#         print(self.id, ": ", self.distance)
        
class Ant(pygame.sprite.Sprite):
    def __init__(self, image_path, speed, start_x, start_y, sw, sh):
        super().__init__()
        self.image = pygame.image.load(image_path).convert()
        self.image.set_colorkey(WHITE)
        self.rect = self.image.get_rect()
        self.rect.center = [start_x, start_y]
        self.speed = speed
        self.start_x = start_x
        self.start_y = start_y
        self.food = False
        self.food_start_x = 0
        self.food_start_y = 0
        self.pher_start_x = 0
        self.pher_start_y = 0
        self.sign_x = 0
        self.sign_y = 0
        self.pos = self.image.get_rect(center=(start_x, start_y))
        self.radius = 35
        self.scent = 0
        self.prev_scent = -1
        self.target = (-1, -1)
        self.end = False
        self.free = False
        self.timer = 100
        self.pheromone_list = []
    
    def update(self, nest_rect, sw, sh, food): # random walk with persistance [adapted from Prof. Ahearn's GAM_3]
        if food == False and self.target == (-1, -1):
            
#             if self.timer == 0:
#                 self.free = False
            
            # horizontal sign
            xsign = np.random.randint(-1, 2)
            p_prob = np.random.randint(0, 10)
            if(p_prob <= 8):
                sign_x = self.sign_x
            else:
                sign_x = xsign          
            # vertical sign   
            ysign = np.random.randint(-1, 2)
            p_prob = np.random.randint(0, 10)
            if(p_prob <= 8): 
                sign_y = self.sign_y 
            else:
                sign_y = ysign  
        
            self.rect.x += (sign_x * self.speed)
            self.rect.y += (sign_y * self.speed)
    
            # keep it on the screen
            if self.rect.right > sw:
                self.rect.left = 0
            if self.rect.bottom > sh:
                self.rect.top = 0
            if self.rect.left < 0:
                self.rect.right = sw
            if self.rect.top < 0:
                self.rect.bottom = sh         
        
            # save previous sign    
            self.sign_x = sign_x
            self.sign_y = sign_y
        
        elif food == False and self.target != (-1, -1):
            xsign = 0
            ysign = 0
            if self.rect.center[0] < self.target[0]:
                xsign = 1         
            elif self.rect.center[0] > self.target[0]:
                xsign = -1
            if self.rect.center[1] < self.target[1]:
                ysign = 1
            elif self.rect.center[1] > self.target[1]:
                ysign = -1
        
            
            self.rect.x += (xsign * self.speed)
            self.rect.y += (ysign * self.speed)
            
            self.sign_x = xsign
            self.sign_y = ysign
        
        #movement back to the nest
        elif food == True:
            sign_x = 0
            sign_y = 0
            if self.rect.x >= nest_rect.right:
                sign_x = -1
            elif self.rect.x <= nest_rect.left:
                sign_x = 1
            if self.rect.y <= nest_rect.top:
                sign_y = 1
            elif self.rect.y >= nest_rect.bottom:
                sign_y = -1
            
            self.rect.x += (sign_x * self.speed)
            self.rect.y += (sign_y * self.speed)
        
        #keep on screen
            if self.pos.right > sw:
                self.pos.left = 0
            if self.pos.bottom > sh:
                self.pos.top = 0
            if self.pos.left < 0:
                self.pos.right = sw
            if self.pos.top < 0:
                self.pos.bottom = sh
            
            self.sign_x = sign_x
            self.sign_y = sign_y
    
#--General functions--

#--General setup--
pygame.init()

#Screen
screen_width = 1000
screen_height = 700
screen = pygame.display.set_mode((screen_width, screen_height))
background = pygame.image.load("bgnd.jpg")

#--Instantiations--

#groups
all_sprites_list = pygame.sprite.RenderUpdates()

ants_with_food = pygame.sprite.RenderUpdates()
pheromones = pygame.sprite.RenderUpdates()

#nest
nest_group = pygame.sprite.RenderUpdates()
nest = Nest()
nest.rect.x = r.randrange(0, screen_width-50)
nest.rect.y = r.randrange(0, screen_height-50)
nest_group.add(nest)
all_sprites_list.add(nest)

#food clusters
food_group = pygame.sprite.RenderUpdates()
for f in range(20):
    new_food = Food()
    new_food.rect.x = r.randrange(0, screen_width-30)
    new_food.rect.y = r.randrange(0, screen_height-30)
    for bit in range(9):
        bit = Food()
        bit.rect.x = new_food.rect.x + r.randrange(0, 30)
        bit.rect.y = new_food.rect.y + r.randrange(0, 30)
        food_group.add(bit)
        all_sprites_list.add(bit)
    food_group.add(new_food)
    all_sprites_list.add(new_food)

#ant
ant = "ant.png"
ant_speed = 1
ant_group = pygame.sprite.RenderUpdates()
for ant in range(100):
    new_ant = Ant("ant.png", ant_speed, np.random.random_sample()*screen_width, np.random.random_sample()*screen_height, screen_width, screen_height)
    ant_group.add(new_ant)
    all_sprites_list.add(new_ant)

#--Runtime--
id = 0 #pheromone id
clock = pygame.time.Clock()
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    screen.blit(background, (0, 0))
    
    ant_group.update(nest.rect, screen_width, screen_height, False)
    ants_with_food.update(nest.rect, screen_width, screen_height, True)
    
    for ant in ants_with_food:
        if math.hypot(ant.pher_start_x - ant.rect.x, ant.pher_start_y - ant.rect.y) > 25:
            id += 1
            ant.pher_start_x = ant.rect.x
            ant.pher_start_y = ant.rect.y
            p_pos_x = ant.rect.x
            p_pos_y = ant.rect.y
            new_p = Pheromone(id, nest.rect, p_pos_x, p_pos_y)
            pheromones.add(new_p)
            all_sprites_list.add(new_p)
    
    for each in pheromones:
        each.timer -= 1
        if each.timer == 0:
            pheromones.remove(each)
            all_sprites_list.remove(each)
            
#     for ant in ant_group:
#         nest_dist = math.hypot(ant.rect.center[0] - nest.rect.center[0], ant.rect.center[1] - nest.rect.center[1])
#         if ant.prev_scent == ant.scent and nest_dist > 40:
# #             print("im free")
# #             ant.free = True
# #             ant.target = (-1, -1)
#             ant.scent = 0
#             ant.prev_scent = -1
    
    hit_dict = pygame.sprite.groupcollide(food_group, ant_group, 1, 0)
    for key, value in hit_dict.items():
#         print("found food!")
        value[0].free = False
        value[0].scent = 0
        value[0].prev_scent = -1
        id += 1
        value[0].pher_start_x = value[0].rect.x
        value[0].pher_start_y = value[0].rect.y
        p_pos_x = value[0].rect.x
        p_pos_y = value[0].rect.y
        new_p = Pheromone(id, nest.rect, p_pos_x, p_pos_y)
        pheromones.add(new_p)
        all_sprites_list.add(new_p)
        ants_with_food.add(value)
        ant_group.remove(value)     
    
    nest_hit = pygame.sprite.groupcollide(nest_group, ants_with_food, 0, 0)
    for key, value in nest_hit.items():
        ant_group.add(value)
        ants_with_food.remove(value)
        
    pher_dict = pygame.sprite.groupcollide(ant_group, pheromones, 0, 0, collided = pygame.sprite.collide_circle)
    for ant, phers in pher_dict.items():
        far_p = max(phers, key=opr.attrgetter("distance"))
#         print(far_p.distance)
        ant.target = far_p.rect.center
        ant.prev_scent = ant.scent
        ant.scent = far_p.id
        if ant.prev_scent == ant.scent:
            ant.target = (-1, -1)
        
#     food_dict = pygame.sprite.groupcollide(ant_group, food_group, 0, 0, collided = pygame.sprite.collide_circle)
#     for ant, foods in food_dict.items():
#         ant.target = foods[0].rect.center
        
#display
    all_sprites_list.clear(screen, background)
    all_sprites_list.draw(screen)
    pygame.display.update()
    clock.tick(60) 
    
pygame.quit()

pygame 2.0.1 (SDL 2.0.14, Python 3.6.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
